In [266]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta
import matplotlib as mpl
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
from datetime import datetime, timedelta
import warnings
import datetime
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)
mpl.style.use('seaborn')
pd.set_option('display.max_columns', 1000)
from sklearn.metrics import accuracy_score


In [267]:
df=pd.read_csv("aud_usd_h4_h1_all_merged_corr_0.1_v1.csv")
df.columns = df.columns.str.replace('.','_')

In [268]:
df.head()

Unnamed: 0                 time_local  target1_x  target2_x  target3_x  \
0           0  2010-01-05 18:00:00+00:00     0.9117     0.9135     0.9142   
1           1  2010-01-05 22:00:00+00:00     0.9135     0.9142     0.9142   
2           2  2010-01-06 02:00:00+00:00     0.9142     0.9142     0.9156   
3           3  2010-01-06 06:00:00+00:00     0.9142     0.9156     0.9184   
4           4  2010-01-06 10:00:00+00:00     0.9156     0.9184     0.9196   

   bid_close_x  Volatility_USD_hourly  PC_macro_economic_ind_USD_hourly  \
0       0.9124                    0.0                          0.000000   
1       0.9117                    0.5                         -1.543434   
2       0.9135                    0.5                         -1.543434   
3       0.9142                    0.5                         -1.543434   
4       0.9142                    0.5                         -1.543434   

   Volatility_USD_daily  PC_macro_economic_ind_USD_daily  \
0                   1.0                         0.051546   
1                   1.0                         0.051546   
2                   1.0                        -1.050000   
3                   1.0                        -1.050000   
4                   1.0                        -1.050000   

   Volatility_AUD_hourly  PC_macro_economic_ind_AUD_hourly  \
0                    0.0                          0.000000   
1                    0.5                         -1.543434   
2                    0.5                         -1.543434   
3                    0.5                         -1.543434   
4                    0.5                         -1.543434   

   Volatility_AUD_daily  PC_macro_economic_ind_AUD_daily  \
0                   1.0                         0.051546   
1                   1.0                         0.051546   
2                   1.0                        -1.050000   
3                   1.0                        -1.050000   
4                   1.0                        -1.050000   

   ANZ Job Advertisements_PC_AUD_hourly  \
0                                   0.0   
1                                   0.0   
2                                   0.0   
3                                   0.0   
4                                   0.0   

   AiG Performance of Mfg Index_PC_AUD_hourly  \
0                                         0.0   
1                                         0.0   
2                                         0.0   
3                                         0.0   
4                                         0.0   

   Building Permits (MoM)_PC_AUD_hourly  Building Permits (YoY)_PC_AUD_hourly  \
0                              0.000000                              0.000000   
1                             -4.484536                              1.397668   
2                             -4.484536                              1.397668   
3                             -4.484536                              1.397668   
4                             -4.484536                              1.397668   

   CFTC AUD NC net positions_PC_AUD_hourly  \
0                                      0.0   
1                                      0.0   
2                                      0.0   
3                                      0.0   
4                                      0.0   

   Construction Work Done_PC_AUD_hourly  \
0                                   0.0   
1                                   0.0   
2                                   0.0   
3                                   0.0   
4                                   0.0   

   Consumer Price Index (QoQ)_PC_AUD_hourly  \
0                                       0.0   
1                                       0.0   
2                                       0.0   
3                                       0.0   
4                                       0.0   

   Dwelling Unit Starts (QoQ)_PC_AUD_hourly  \
0                                       0.0   
1                                       

In [269]:
df=df.dropna(how='all',axis=1)
df.shape

(14028, 839)

In [270]:
df['time_local']=df['time_local'].astype(str).str[:-6]

In [271]:
df['year']=pd.DatetimeIndex(df['time_local']).year
df['month']=pd.DatetimeIndex(df['time_local']).month
df['day']=pd.DatetimeIndex(df['time_local']).day
df['hour']=pd.DatetimeIndex(df['time_local']).hour
df['dayofweek']=pd.DatetimeIndex(df['time_local']).dayofweek



In [272]:
import datetime   
#date = datetime.date.today()
date=pd.datetime.now().strftime("%Y-%m-%d %I:%M:%S")
date

'2019-04-25 12:42:46'

In [273]:
df['today']=date
df['today'] =pd.DatetimeIndex(df['today'])

In [274]:
time_local=df[(df['year']==2018) | (df['year']==2019)]['time_local']
time_local.to_csv("time_local_v1_two_class.csv")

In [275]:
df['nb_months'] = ((df['today'] - pd.to_datetime(df['time_local'].astype(str).str[:-6]))/np.timedelta64(1, 'M'))
df['nb_months'] = df['nb_months'].astype(int)

In [276]:
df['class_target_7'] = ((df['target1_x']-df['close_h1_7'])/df['close_h1_7'])*100
df.loc[df['class_target_7'] < 0.0005, 'new_class_target'] = -1
df.loc[df['class_target_7'] > 0.0005, 'new_class_target'] = 1
df.loc[df['class_target_7'] == 0, 'new_class_target'] = 0
#class_target3

In [277]:
df['new_class_target'].value_counts()

-1.0    6624
 1.0    6579
 0.0     825
Name: new_class_target, dtype: int64

In [278]:
col_drop= df.filter(regex='time').columns

In [279]:
target_cols = df.filter(regex='target').columns

In [280]:
cols_x = df.filter(regex='_x').columns

In [281]:
cols_y = df.filter(regex='_y').columns

In [282]:
X=df.drop(['Unnamed: 0','time_local', 'target2_x', 'target3_x', 'target1_y', 'target2_y',
       'target3_y', 'class_target2', 'class_target3',
       'PC_AUD_H4_target1', 'bid_close_y', 'today','target1_x','class_target1'],axis=1)

In [283]:
X=X.fillna(0)

In [284]:
df_temp=X.select_dtypes(include=['object'])

In [285]:
obj=df_temp.loc[:, df_temp.dtypes =='object']
for col in obj:
    obj[col] = pd.to_numeric(obj[col], errors='coerce')
df_temp[obj.columns]=obj

In [286]:
df_temp=df_temp.replace([np.inf, -np.inf], np.nan)
df_temp=df_temp.fillna(0)
X[df_temp.columns]=df_temp

In [287]:
#X_final= X[X['year'] < 2018]
s=X[['year','month', 'dayofweek']]
for each in s:
    dummies = pd.get_dummies(s[each], prefix=each, drop_first=False)
    X=X.drop([each],axis=1)
    X = pd.concat([X, dummies], axis=1)

In [288]:
X['target_2_class_1']=np.where(X['new_class_target']==-1,-1,0)
X['target_2_class_2']=np.where(X['new_class_target']==1,1,0)
X=X[X.replace([np.inf, -np.inf], np.nan).notnull().all(axis=1)]
X_final=X[(X['year_2019']!=1) & (X['year_2018']!=1) ]



In [289]:
X_pred=X[(X['year_2019']==1) | (X['year_2018']==1)]
X_pred.to_csv("X_close_7_2018_25_4_2019_two_class.csv")

In [211]:
y=X_final['target_2_class_1']
X1=X_final.drop(['target_2_class_1','target_2_class_2','class_target_7','new_class_target'],axis=1)

In [212]:
X_final['target_2_class_1'].value_counts()

 0    6349
-1    5736
Name: target_2_class_1, dtype: int64

In [213]:
#from sklearn.preprocessing import StandardScaler
#scaler=StandardScaler()
#scaler.fit(X1)
#X1=scaler.transform(X1)

In [214]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
np.seed=0
from sklearn.linear_model import LogisticRegression

In [215]:
from imblearn.over_sampling import SMOTE
smt = SMOTE(random_state=1)
X2, y1 = smt.fit_sample(X1, y)
len(X2),len(y1)

(12698, 12698)

In [250]:
X_train, X_test, y_train, y_test = train_test_split(X2, y1, test_size=0.3,random_state=1,stratify=y1)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)


(8888, 857) (8888,)
(3810, 857) (3810,)


In [217]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier()
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)

In [218]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.53      0.52      0.52      1905
           0       0.53      0.54      0.53      1905

   micro avg       0.53      0.53      0.53      3810
   macro avg       0.53      0.53      0.53      3810
weighted avg       0.53      0.53      0.53      3810



In [219]:
print(classification_report(y_train,gb.predict(X_train)))

              precision    recall  f1-score   support

          -1       0.73      0.74      0.73      4444
           0       0.74      0.72      0.73      4444

   micro avg       0.73      0.73      0.73      8888
   macro avg       0.73      0.73      0.73      8888
weighted avg       0.73      0.73      0.73      8888



In [220]:
accuracy_score(y_test,y_pred)

0.5272965879265092

In [221]:
accuracy_score(y_train,gb.predict(X_train))

0.7318856885688569

In [222]:
feature_importance=pd.DataFrame(gb.feature_importances_)
feature_importance.columns=['Feature_importance']
feature_cols=pd.DataFrame(X1.columns)
feature_importance=pd.concat([feature_cols,feature_importance],axis=1).sort_values(by='Feature_importance',ascending=False)

In [223]:
feature_importance.head(10)

0  Feature_importance
827                    hour            0.063867
855             dayofweek_4            0.039515
371       correl_EUR_AUD_7p            0.021521
364        CS_AUD_GBP_H4_20            0.021470
798    STDEV_AUD_H1_2p_h1_4            0.018235
822         PC3_AUD_H1_h1_7            0.016908
828               nb_months            0.014434
376             TR_ADX_AUD4            0.013954
836               year_2017            0.013511
808  correl_AUD_CHF_3p_h1_5            0.012959

In [ ]:
pd.crosstab(y_test,y_pred)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.8,n_estimators=80)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)

In [ ]:
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.1,subsample=0.8,n_estimators=120)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.1,subsample=0.8,n_estimators=90)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.1,subsample=0.6,n_estimators=100)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.6,n_estimators=100)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.6,n_estimators=100,max_depth=4)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

# Final model

In [251]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.4,n_estimators=60,max_depth=2,max_features ="auto",n_iter_no_change=3,random_state=1)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.5047244094488189, 0.5447794779477948)

In [252]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.50      0.63      0.56      1905
           0       0.51      0.38      0.43      1905

   micro avg       0.50      0.50      0.50      3810
   macro avg       0.51      0.50      0.50      3810
weighted avg       0.51      0.50      0.50      3810



In [253]:
print(classification_report(y_train,gb.predict(X_train)))

              precision    recall  f1-score   support

          -1       0.54      0.67      0.60      4444
           0       0.56      0.42      0.48      4444

   micro avg       0.54      0.54      0.54      8888
   macro avg       0.55      0.54      0.54      8888
weighted avg       0.55      0.54      0.54      8888



In [254]:
pd.crosstab(y_test,y_pred)

col_0    -1    0
row_0           
-1     1197  708
 0     1179  726

In [255]:
pd.crosstab(y_train,gb.predict(X_train))

col_0    -1     0
row_0            
-1     2996  1448
 0     2598  1846

In [256]:
from joblib import dump, load
dump(gb,'target_2_class_1_grad_close_7.joblib') 

['target_2_class_1_grad_close_7.joblib']

In [229]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,model.predict(X_train))

(0.526246719160105, 0.7247974797479748)

In [230]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          -1       0.53      0.52      0.52      1905
           0       0.53      0.53      0.53      1905

   micro avg       0.53      0.53      0.53      3810
   macro avg       0.53      0.53      0.53      3810
weighted avg       0.53      0.53      0.53      3810



In [231]:
print(classification_report(y_train,gb.predict(X_train)))

              precision    recall  f1-score   support

          -1       0.64      0.65      0.64      4444
           0       0.64      0.63      0.64      4444

   micro avg       0.64      0.64      0.64      8888
   macro avg       0.64      0.64      0.64      8888
weighted avg       0.64      0.64      0.64      8888



In [232]:
from xgboost import XGBClassifier
model = XGBClassifier(eta=0.01,subsample=0.8,colsample_bytree=0.8)
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,model.predict(X_train))

(0.5335958005249344, 0.7273852385238524)

In [237]:
from xgboost import XGBClassifier
model = XGBClassifier(eta=0.01,subsample=0.6,colsample_bytree=0.6,reg_lambda=2)
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,model.predict(X_train))

(0.5286089238845144, 0.7134338433843385)

In [257]:
y=X_final['target_2_class_2']
X1=X_final.drop(['target_2_class_1','target_2_class_2','class_target_7','new_class_target'],axis=1)

In [258]:
X_train, X_test, y_train, y_test= train_test_split(X1, y,test_size=0.3,random_state=1,stratify=y)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(8459, 857) (8459,)
(3626, 857) (3626,)


In [259]:
from xgboost import XGBClassifier
model = XGBClassifier(reg_lambda=1)
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,model.predict(X_train))

(0.5068946497517927, 0.6966544508807188)

In [260]:
feature_importance=pd.DataFrame(model.feature_importances_)
#feature_importance.columns=['Feature_importance']
#feature_cols=pd.DataFrame(X1.columns)
#feature_importance=pd.concat([feature_cols,feature_importance],axis=1).sort_values(by='Feature_importance',ascending=False)

In [181]:
feature_importance.head()

0
0 NaN
1 NaN
2 NaN
3 NaN
4 NaN

In [262]:
from sklearn.ensemble import GradientBoostingClassifier
gb1=GradientBoostingClassifier(learning_rate=0.01,subsample=0.6,n_estimators=120,max_depth=3,random_state=1)
gb1.fit(X_train,y_train)
y_pred=gb1.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb1.predict(X_train))

(0.5308880308880309, 0.5585766639082634)

In [264]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.53      0.97      0.69      1924
           1       0.50      0.03      0.06      1702

   micro avg       0.53      0.53      0.53      3626
   macro avg       0.52      0.50      0.37      3626
weighted avg       0.52      0.53      0.39      3626



In [263]:
print(classification_report(y_train,gb1.predict(X_train)))

              precision    recall  f1-score   support

           0       0.55      1.00      0.71      4490
           1       0.93      0.06      0.12      3969

   micro avg       0.56      0.56      0.56      8459
   macro avg       0.74      0.53      0.41      8459
weighted avg       0.72      0.56      0.43      8459



In [290]:
dump(gb1,'target_2_class_2_gb_close_7.joblib') 

['target_2_class_2_gb_close_7.joblib']